The aim of this script is to simulate the motion a chaser satellite in close proximity to a target satellite. This uses the Clohessy Wiltshire equations to draw the chaser satellite in the reference frame of the target satellite. This particular sumulation attempts to simulate the motion of the satellite given inital condistions and then provides the means to apply an extended burn at a across a chosen time period and fuel capacity. 


This script works on making the fuel compoent of the siluation more accurate.

In [7]:
import numpy as np
import plotly.graph_objects as go
import math

# Gravitational parameter (mu) and mean motion (n)
mu = 398600.4418  # km^3/s^2 (gravitational parameter of Earth)
sma = 7000  # km (semi-major axis of the target's orbit)
n = np.sqrt(mu / sma**3)  # mean motion of the target

# Initial conditions in the target's orbit reference frame
r0_chaser = np.array([10.0, 9.0, -30.0])  # Initial relative position of chaser, km
v0_chaser = np.array([3, 5, 0.1])  # Initial relative velocity of chaser, km/s

# Thrust parameters
thrust_start_time = 2000  # Time delay before thrust starts in seconds
thrust_duration = 6000    # Thrust duration in seconds
thrust_force = np.array([0, 100000, 0])  # Thrust force in N
specific_impulse = 3000  # Specific impulse of the propulsion system, seconds
fuel_mass_inital = 1000  # Initial mass of the fuel
spacecraft_mass = 4000 # Initial dry mass of the craft, kg
total_mass_initial = spacecraft_mass + fuel_mass_inital  # Initial total mass of the spacecraft

# Time parameters
t_final = 10000  # Total simulation time, seconds
dt = 1  # Time step, seconds

# Time array
t = np.arange(0, t_final, dt)

# Position and velocity arrays
r_chaser = np.zeros((len(t), 3))
v_chaser = np.zeros((len(t), 3))

# Set initial conditions
r_chaser[0] = r0_chaser
v_chaser[0] = v0_chaser

# Clohessy-Wiltshire equations for relative motion
def clohessy_wiltshire(r, v, t, n):
    x, y, z = r
    x_dot, y_dot, z_dot = v
    
    x_ddot = 3 * n**2 * x + 2 * n * y_dot
    y_ddot = -2 * n * x_dot
    z_ddot = -n**2 * z
    
    return np.array([x_dot, y_dot, z_dot]), np.array([x_ddot, y_ddot, z_ddot])


# Initialize the flag variable before the loop
fuel_depleted_flag = False

#initialise current mass
current_mass = total_mass_initial

# Simulation loop
for i in range(1, len(t)):
    # Get current state
    r = r_chaser[i-1]
    v = v_chaser[i-1]
    
    # Find accelerations using Clohessy-Wiltshire equations
    v, a = clohessy_wiltshire(r, v, t[i], n)
    thrust_acceleration = np.array([0.0, 0.0, 0.0])
    # Calculate thrust acceleration using rocket equation
    if thrust_start_time <= t[i] <= (thrust_start_time + thrust_duration):
    
        if (current_mass - spacecraft_mass) > 0:
            exhaust_velocity = specific_impulse * 9.81  # Exhaust velocity in m/s
            thrust_acceleration = thrust_force / current_mass  # Thrust acceleration in m/s^2
            thrust_acceleration = thrust_acceleration * 1e-3  # Convert thrust acceleration to km/s^2
            print("Thrust acceleration (km/s^2):", thrust_acceleration)
            current_mass = current_mass*math.exp((-(np.linalg.norm(thrust_acceleration)*dt))/(exhaust_velocity))  # Update mass of the spacecraft
        else:
            if not fuel_depleted_flag:
                fueldepleted = True
                print("fuel depleted at: ", t[i], "seconds")
                fuel_depleted_flag = True
            thrust_acceleration = np.array([0.0, 0.0, 0.0])  # No thrust
    
    # Total acceleration
    total_acceleration = a + thrust_acceleration
    
    # Update position and velocity
    r_chaser[i] = r + v * dt
    v_chaser[i] = v + total_acceleration * dt


print("fuel used: ", (fuel_mass_inital-(current_mass - spacecraft_mass)) , "kg, out of ", fuel_mass_inital, "kg")




# Plotting with Plotly for interactive visualization
r_chaser_scaled = r_chaser / sma

# Create traces for plotting
trace = go.Scatter3d(
    x=r_chaser_scaled[:, 0],
    y=r_chaser_scaled[:, 1],
    z=r_chaser_scaled[:, 2],
    mode='markers',
    name='Chaser Satellite',
    marker=dict(
        color='blue',
        size=2
    )
)

initial_trace = go.Scatter3d(
    x=[r_chaser_scaled[0, 0]],
    y=[r_chaser_scaled[0, 1]],
    z=[r_chaser_scaled[0, 2]],
    mode='markers',
    name='Initial Position',
    marker=dict(
        color='black',
        size=5
    )
)

target_trace = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    name='Target Satellite',
    marker=dict(
        color='red',
        size=5
    )
)

thrust_start_trace = go.Scatter3d(
    x=[r_chaser_scaled[thrust_start_time, 0]],
    y=[r_chaser_scaled[thrust_start_time, 1]],
    z=[r_chaser_scaled[thrust_start_time, 2]],
    mode='markers',
    name='Thrust Start',
    marker=dict(
        color='green',
        size=5
    )
)

if (current_mass - spacecraft_mass) <= 0:
    thrust_end_trace = go.Scatter3d(
        x=[r_chaser_scaled[fueldepleted, 0]],
        y=[r_chaser_scaled[fueldepleted, 1]],
        z=[r_chaser_scaled[fueldepleted, 2]],
        mode='markers',
        name='Thrust End',
        marker=dict(
            color='yellow',
            size=5
        )
    )
else:
    thrust_end_trace = go.Scatter3d(
        x=[r_chaser_scaled[thrust_start_time+thrust_duration, 0]],
        y=[r_chaser_scaled[thrust_start_time+thrust_duration, 1]],
        z=[r_chaser_scaled[thrust_start_time+thrust_duration, 2]],
        mode='markers',
        name='Thrust End',
        marker=dict(
            color='yellow',
            size=5
        )
    )

# Create the layout
layout = go.Layout(
    title='Relative Motion of the Chaser Satellite',
    scene=dict(
        xaxis=dict(title='X/a [km]'),
        yaxis=dict(title='Y/a [km]'),
        zaxis=dict(title='Z/a [km]')
    )
)

# Create the figure and add the traces
fig = go.Figure(data=[trace, target_trace, thrust_start_trace, thrust_end_trace, initial_trace], layout=layout)

# Show the figure
fig.show()


Thrust acceleration (km/s^2): [0.   0.02 0.  ]
Thrust acceleration (km/s^2): [0.         0.02000001 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000003 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000004 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000005 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000007 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000008 0.        ]
Thrust acceleration (km/s^2): [0.        0.0200001 0.       ]
Thrust acceleration (km/s^2): [0.         0.02000011 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000012 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000014 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000015 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000016 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000018 0.        ]
Thrust acceleration (km/s^2): [0.         0.02000019 0.        ]
Thrust acceleration (km/s^2): [0.        0.020